# Hybrid Recommender System

### İş Problemi

ID'si verilen kullanıcı için item-based ve user-based recommender
yöntemlerini kullanarak 10 film önerisi yapınız.

### Veri Seti Hikayesi

Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan
derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016
tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar
rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.
1. movie.csv
- movieId: Eşsiz film numarası
- title: Film adı
- genres: Tür
2. rating.csv
- userid: Eşsiz kullanıcı numarası (UniqueID)
- movieId: Eşsiz film numarası (UniqueID)
- rating: Kullanıcı tarafından filme verilen puan
- timestamp Değerlendirme tarihi

## User Based Recommendation

### Veri Hazırlama

In [1]:
import pandas as pd

In [2]:
rating_df = pd.read_csv("D:\\Recommendation Systems\\hybrid_recommender\\datasets\\rating.csv")
movie_df = pd.read_csv("D:\\Recommendation Systems\\hybrid_recommender\\datasets\\movie.csv")
df = movie_df.merge(rating_df, how="left", on="movieId")

In [3]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [4]:
comments = df["title"].value_counts()
# toplam oy sayısı 

In [5]:
comments = pd.DataFrame(comments)

In [6]:
comments

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [7]:
low_films = comments[comments["title"] < 1000]
# oy sayısı 1000'den az filmler

In [8]:
low_films

,title
"Bear, The (Ours, L') (1988)",999
Rosewood (1997),999
Ted (2012),999
One Night at McCool's (2001),999
Marked for Death (1990),998
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [10]:
movies = df[~df["title"].isin(low_films.index)]

In [11]:
movies["title"].value_counts().sort_values(ascending=True)

Wristcutters: A Love Story (2006)     1001
Lincoln Lawyer, The (2011)            1001
Pet Sematary II (1992)                1003
Return to Paradise (1998)             1003
Scanners (1981)                       1003
                                     ...  
Jurassic Park (1993)                 59715
Silence of the Lambs, The (1991)     63299
Shawshank Redemption, The (1994)     63366
Forrest Gump (1994)                  66172
Pulp Fiction (1994)                  67310
Name: title, Length: 3159, dtype: int64

In [12]:
movies.shape

(17766015, 6)

In [13]:
movies

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


In [14]:
user_movie_df = movies.pivot_table(index="userId", columns=["title"], values="rating")

In [15]:
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi

In [16]:
random_user = 108170
# random kullanıcı

In [17]:
random_user_df = user_movie_df.loc[user_movie_df.index == random_user]
# random kullanıcıya ait gözlem birimleri

In [18]:
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
108170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
movies_watched = random_user_df.columns[random_user_df.notna().any().to_list()]
# seçilen kullanıcının izlediği filmler

In [20]:
movies_watched

Index(['2001: A Space Odyssey (1968)',
       'Adventures of Priscilla, Queen of the Desert, The (1994)',
       'Akira (1988)', 'Aladdin (1992)', 'Aliens (1986)',
       'Almost Famous (2000)', 'Along Came Polly (2004)', 'Amadeus (1984)',
       'American History X (1998)', 'Any Given Sunday (1999)',
       ...
       'Untouchables, The (1987)', 'Usual Suspects, The (1995)',
       'Wallace & Gromit: A Close Shave (1995)',
       'Wallace & Gromit: The Best of Aardman Animation (1996)',
       'Wallace & Gromit: The Wrong Trousers (1993)',
       'Wedding Singer, The (1998)', 'Who Framed Roger Rabbit? (1988)',
       'Wild at Heart (1990)', 'Willow (1988)', 'X2: X-Men United (2003)'],
      dtype='object', name='title', length=186)

### Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişilmesi

In [21]:
movies_watched_df = user_movie_df[movies_watched]
# seçilen kullanıcının izlediği filmlere göre filtreleme

In [22]:
movies_watched_df

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Wild at Heart (1990),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.5,4.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
user_movie_count = movies_watched_df.T.notnull().sum()
# her bir kullancının seçili user'in izlediği filmlerin kaçını izlediğini bilgisi

In [24]:
user_movie_count = user_movie_count.reset_index()

In [25]:
user_movie_count.columns = ["userId", "movie_count"]

In [26]:
user_movie_count

,userId,movie_count
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16
...,...,...
138488,138489.0,11
138489,138490.0,10
138490,138491.0,3
138491,138492.0,20


In [27]:
perc = len(movies_watched) * 60 / 100

In [28]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
# seçilen kullanıcının oy verdiği filmlerin yüzde 60 ve üstünü izleyenlerin kullanıcı id’leri

In [29]:
users_same_movies

90            91.0
115          116.0
155          156.0
293          294.0
297          298.0
            ...   
138133    138134.0
138207    138208.0
138253    138254.0
138324    138325.0
138410    138411.0
Name: userId, Length: 2326, dtype: float64

### Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi

In [30]:
user_movie_df[movies_watched]

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Wild at Heart (1990),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.5,4.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]

In [32]:
final_df
# seçili kullanıcı ile benzerlik gösteren kullanıcıların id’leri

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),...,"Untouchables, The (1987)","Usual Suspects, The (1995)",Wallace & Gromit: A Close Shave (1995),Wallace & Gromit: The Best of Aardman Animation (1996),Wallace & Gromit: The Wrong Trousers (1993),"Wedding Singer, The (1998)",Who Framed Roger Rabbit? (1988),Wild at Heart (1990),Willow (1988),X2: X-Men United (2003)
userId,,,,,,,,,,,,,,,,,,,,,
91.0,2.5,4.0,2.5,NaN,4.0,3.0,NaN,3.5,3.5,NaN,...,4.0,3.5,4.0,4.0,4.0,3.5,4.0,NaN,3.5,3.5
116.0,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.5,3.5,...,NaN,4.5,NaN,NaN,3.5,2.0,3.0,NaN,1.0,4.5
156.0,NaN,3.0,NaN,NaN,5.0,5.0,NaN,4.0,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN
294.0,4.5,4.0,4.5,4.0,4.5,3.5,NaN,2.5,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,5.0,NaN,4.5,3.5
298.0,NaN,NaN,NaN,3.0,5.0,5.0,NaN,NaN,4.0,5.0,...,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138134.0,5.0,NaN,NaN,4.0,3.0,4.5,NaN,4.0,4.5,NaN,...,3.0,NaN,NaN,NaN,4.0,NaN,3.0,NaN,NaN,3.5
138208.0,3.0,3.5,NaN,4.0,3.0,4.0,NaN,3.0,4.0,NaN,...,3.0,4.0,3.0,3.0,NaN,3.0,3.0,2.5,3.0,3.0
138254.0,NaN,NaN,NaN,3.5,4.0,NaN,NaN,NaN,4.5,NaN,...,4.5,4.5,NaN,NaN,NaN,3.0,4.0,NaN,3.5,4.0


In [33]:
corr_df = final_df.T.corr().unstack()
# kullanıcıların birbirleri ile olan korelasyonları

In [34]:
corr_df = pd.DataFrame(corr_df,columns=["corr"])

In [35]:
corr_df

corr
userId   userId            
91.0     91.0      1.000000
         116.0     0.454166
         156.0     0.322393
         294.0     0.338482
         298.0     0.152716
...                     ...
138411.0 138134.0  0.361395
         138208.0  0.449004
         138254.0  0.308810
         138325.0  0.473589
         138411.0  1.000000

[5410276 rows x 1 columns]

In [36]:
corr_df.index.names = ['user_id_1', 'user_id_2']

In [37]:
corr_df = corr_df.reset_index()

In [38]:
corr_df

,user_id_1,user_id_2,corr
0,91.0,91.0,1.000000
1,91.0,116.0,0.454166
2,91.0,156.0,0.322393
3,91.0,294.0,0.338482
4,91.0,298.0,0.152716
...,...,...,...
5410271,138411.0,138134.0,0.361395
5410272,138411.0,138208.0,0.449004
5410273,138411.0,138254.0,0.308810
5410274,138411.0,138325.0,0.473589


In [39]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]]
# seçili kullanıcı ile yüksek korelasyona sahip (0.65’in üzerinde olan) kullanıcıları filtrelenmesi

In [40]:
top_users.reset_index(drop=True)

,user_id_2,corr
0,5155.0,0.716406
1,11517.0,0.707652
2,12710.0,0.663332
3,82860.0,0.654646
4,108170.0,1.000000
5,119285.0,0.700081
6,121747.0,0.673295


In [43]:
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [44]:
top_users_ratings = top_users.merge(rating_df[["userId", "movieId", "rating"]], how='inner')

In [45]:
top_users_ratings

,userId,corr,movieId,rating
0,5155.0,0.716406,1,3.5
1,5155.0,0.716406,2,3.0
2,5155.0,0.716406,5,3.0
3,5155.0,0.716406,9,3.0
4,5155.0,0.716406,10,4.0
...,...,...,...,...
4752,121747.0,0.673295,27846,4.5
4753,121747.0,0.673295,30707,4.5
4754,121747.0,0.673295,30810,4.5
4755,121747.0,0.673295,31221,2.0


In [46]:
top_users_ratings["userId"].unique()

array([  5155.,  11517.,  12710.,  82860., 108170., 119285., 121747.])

### Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması

In [47]:
top_users_ratings["weighted_rating"] =  top_users_ratings["corr"] * top_users_ratings["rating"]

In [49]:
recommendation_df = top_users_ratings.groupby("movieId").agg({"weighted_rating":"mean"})

In [50]:
recommendation_df

,weighted_rating
movieId,
1,2.249311
2,1.319527
3,0.986312
5,1.484503
6,2.526375
...,...
113573,1.769131
116823,1.769131
116887,2.122957


In [54]:
recommendation_df.loc[recommendation_df["weighted_rating"] >3.5].sort_values("weighted_rating",ascending=False)
# recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmler

,weighted_rating
movieId,
7044,5.000000
2871,5.000000
4125,4.500000
8970,4.500000
27706,4.500000
8039,4.000000
7743,4.000000
2173,4.000000
223,3.870621


In [57]:
rec_films = recommendation_df.loc[recommendation_df["weighted_rating"] >3.5].sort_values("weighted_rating",ascending=False).head()

In [86]:
rec_films
# recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan ilk 5 film

,weighted_rating
movieId,
7044,5.0
2871,5.0
4125,4.5
8970,4.5
27706,4.5


In [87]:
movie_df[movie_df["movieId"].isin(rec_films.index)]
# önerilen filmler

,movieId,title,genres
2785,2871,Deliverance (1972),Adventure|Drama|Thriller
4031,4125,Leonard Part 6 (1987),Comedy|Sci-Fi
6932,7044,Wild at Heart (1990),Crime|Drama|Mystery|Romance|Thriller
8287,8970,Finding Neverland (2004),Drama
9437,27706,Lemony Snicket's A Series of Unfortunate Event...,Adventure|Children|Comedy|Fantasy


## Item Based Recommendation

In [189]:
rating_df = pd.read_csv("D:\\Recommendation Systems\\hybrid_recommender\\datasets\\rating.csv")
movie_df = pd.read_csv("D:\\Recommendation Systems\\hybrid_recommender\\datasets\\movie.csv")

In [190]:
rating_df

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [191]:
user = 108170

In [192]:
movie_id = rating_df[(rating_df["userId"] == user) & (rating_df["rating"] == 5.0)].sort_values("timestamp",ascending=False)["movieId"][0:1].values[0]

In [193]:
movie_id

7044

In [194]:
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
movie_title = movie_df[movie_df["movieId"] == movie_id]["title"].values[0]
# user_movie_df film isimlerinden oluştuğu için filmin ismini çektik

In [196]:
movie_title

'Wild at Heart (1990)'

In [197]:
df_movie = user_movie_df[movie_title]

In [199]:
df_movie

userId
1.0        NaN
2.0        NaN
3.0        NaN
4.0        NaN
5.0        NaN
            ..
138489.0   NaN
138490.0   NaN
138491.0   NaN
138492.0   NaN
138493.0   NaN
Name: Wild at Heart (1990), Length: 138493, dtype: float64

In [203]:
user_movie_df.corrwith(df_movie).sort_values(ascending=False).head()

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
dtype: float64